In [8]:
import astropy.io.fits as fits
import numpy as np
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy.optimize import curve_fit
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, join
import os
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import halomod as hm
import hmf
import treecorr







In [9]:
import DarkVerse as dv


In [10]:

# Define the home directory
home_dir = os.path.expanduser('~')
thesis_path = os.path.join(home_dir, 'Master_Thesis', 'DATA')

# Load Catalog Data
fits_file_path = os.path.join(thesis_path, "Y3_deep_fields_DB_wKNN_cat_COSMOS_zm.fits")  
t = Table.read(fits_file_path)

masked_file_path = os.path.join(thesis_path, "COSMOS_masked_cat.fits")  
t3 = Table.read(masked_file_path)

# Join tables on 'id'
t = join(t, t3, keys='id')
t.rename_column('ra_1', 'ra')
t.rename_column('dec_1', 'dec')

# Load Randoms
fits_random = os.path.join(thesis_path, "COSMOS_randoms_ugriz_trim_video.fits") 
t2 = Table.read(fits_random)
randoms = t2  # Random catalog

# Define Correlation Function Configuration
config = {
    'min_sep': 0.003,
    'max_sep': 1.78,
    'bin_size': 0.1,
    'sep_units': 'degrees',
    'var_method': 'bootstrap'
}

# Define Redshift and Stellar Mass Limits
z_min, z_max = 0.2, 0.4
SM_min, SM_max = 1e10, 5e10

# Apply Subsample
subsample = dv.Subsample(catalog=t, z_min=z_min, z_max=z_max, SM_min=SM_min, SM_max=SM_max)

# Get Results
results = subsample.get_results()

# Print Results
print(f"Power Law Parameters (A, gamma): {results['power_law_params']}")
print(f"w_theta: {results['w_theta']}")
print(f"DD counts: {results['dd_counts']}")
print(f"DR counts: {results['dr_counts']}")
print(f"RR counts: {results['rr_counts']}")


ValueError: Input arrays have zero length